In [62]:
import glob
import os
import numpy as np

import sys
sys.path.append("/home/shoinoue/rsc/projects/sho-meta/pyfiles/")
from json_ext import load_json

base_dir = "./audiosamples/"

In [24]:
webtitle = "Emotion-Aware DialogTTS Demo Page"
title = "Emotion-Aware DialogTTS: Generating Empathetic Speech Dialogs with Turn-Level Emotion Conditioning"
abstract = "abstract"
github_url = "https://github.com/shinshoji01"
# base_repo_dir = "/"
base_repo_dir = "/EDTTS-Demo/"
style_dir = base_repo_dir + "statics/"
fig_path = base_repo_dir + "images/inference.png"

initial = f"""
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <meta content="IE=edge" http-equiv="X-UA-Compatible">
    <meta content="width=device-width, initial-scale=1" name="viewport">
    <title>{webtitle}</title>
    <link href="{style_dir}bootstrap-5.2.3-dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="{style_dir}my.css" rel="stylesheet">
  </head>
  <body>
    <div class="container">
      <div class="row">
        <div class="container pt-5 mt-5 shadow p-5 mb-5 bg-white rounded">
          <div class="text-center">
            <h2>{title}</h2>
            <br>
            <h5 class="mb-4">Sho Inoue<sup>1,*</sup>, Min-Jae Hwang<sup>1</sup></h5>
              <p>
                <sup>1</sup>FAIR at Meta<br>
                <sup>*</sup>Work done at Meta<br>
              </p>
          </div>
          <br>
          <figure class="text-center">
            <img src="{fig_path}" alt="overall diagram of the pipeline" class="img-fluid" style="width: 900px; height: auto;">
          </figure>
          <br>
          <h3>Abstract</h3>
          <p class="lead">
          {abstract}
          </p>
          <p class="lead">You can visit the project page of this paper: <a href="{github_url}">Github Repository</a>.
          </p>
        </div>
"""[1:]

closure = f"""
      </div>
    </div>
    <script src="{style_dir}jquery/jquery-3.7.1.slim.min.js"></script>
    <script src="{style_dir}bootstrap-5.2.3-dist/bootstrap.min.js"></script>
"""[1:]
closure += """
  </body>
  <script>
    $(function(){
        $("audio").on("play", function() {
            $("audio").not(this).each(function(index, audio) {
                audio.pause();
                audio.currentTime = 0;
            });
        });
    });
    </script>
</html>
"""[1:]

In [43]:
experiments = {
    "syn": {
        "dialog/prompt": "Prompt",
        "dialog/baseline---prompt": "Baseline (DTTS)",
        "dialog/textmspvad---vad-linear-regression": "EDTTS",
        "dialog/textmspmanualvad---vad-linear-regression": "EDTTS 2",
        "dialog/cosyvoice": "CosyVoice",
        "dialog/mooncast": "MoonCast",
    },
}

titles = {
    "syn": "Empathetic Dialog Generation (Synthetic Turns)",
}

explanations = {
    "syn": "\
This section includes audio samples generated by our proposed generation method, Emotion-Aware DialogTTS (EDTTS). \
<ul> \
  <li><b>Prompt</b>: American speaker from CMU-ARCTIC <br></li> \
  <li><b>Baseline</b>: Hindi speaker from L2-ARCTIC <br></li> \
</ul> \
sample \
",
}

In [44]:
base_model = list(experiments[list(experiments.keys())[0]].keys())[0]
files = glob.glob(base_dir+f"{base_model}/*/*/*/*.wav")
filenames = list(set([p[len(base_dir)+len(base_model)+1:] for p in files]))
filenames.sort()
np.random.seed(0)
filenames = [str(p) for p in list(np.random.choice(filenames, len(filenames), replace=False))]
filenames_dir = {
    "syn": [fn for fn in filenames if not("davidaitest" in fn)],
}

In [96]:
extexts = {}
for exid in experiments:
    text = f"""
            <div class="container pt-5 mt-5 shadow p-5 mb-5 bg-white rounded">
              <h3>{titles[exid]}</h3>
              <p class="lead">
                    {explanations[exid]}
                    </p>
              <div class="table-responsive" style="overflow-x: scroll">
                <table class="table table-sm">
    """[1:]
    text += f"""
                  <thead>
                    <tr>
                      <th scope="col">ID</th>
    """[1:]
    for fn in filenames_dir[exid]:
        text += f"""
                      <th scope="col">{"/".join(fn.split("/")[:3])}</th>
    """[1:]
    text += """
                    </tr>
                  </thead>
    """[1:]
    headerfn = text

    text = """
                  <tbody>
                    <tr>
                      <th scope="row" style="position: sticky; left: 0; z-index:10; opacity: 1.0; background-color: white;">Text</th>
    """[1:]
    for fn in filenames_dir[exid]:
        # data = np.load(transcription_dir + f"{fn}.npy", allow_pickle=True).item()
        text += f"""
                  <td>
                    <p>{html_table}</p>
                  </td>
    """[1:]
    text += """
                    </tr>
    """[1:]
    headertext = text

    text = ""
    for key in experiments[exid]:
        text += f"""
                    <tr>
                      <th scope="row" style="position: sticky; left: 0; z-index:10; opacity: 1.0; background-color: white;">{experiments[exid][key]}</th>
    """[1:]
        for fn in filenames_dir[exid]:
            wavfile = base_dir + f"{key}/{fn}"
            text += f"""
                      <td>
    """[1:]
            text += f"""
                        <audio controls="" preload="none" style="width: 240px">
                          <source src="{wavfile}" type="audio/wav">
                        </audio>
    """[1:]
            text += f"""
                      </td>
    """[1:]
        text += """
                    </tr>
    """[1:]
    body = text

    text = """
                  </tbody>
                </table>
              </div>
              <p class="lead">* please scroll horizontally to explore additional columns in the table.</p>
            </div>
    """[1:]
    tableclosure = text
    extexts[exid] = headerfn + headertext + body + tableclosure
    extexts[exid] = "\n".join([a[4:] for a in extexts[exid].split("\n")])

In [97]:
html_table = "<table>"
for turnkey in dialog:
    turn = dialog[turnkey]
    html_table += "<tr>"
    html_table += f'<td>{turn["speaker"]}</td>'
    html_table += f'<td>{turn["emotion"]}</td>'
    html_table += f'<td>{turn["transcription"]}</td>'
    html_table += "</tr>"
html_table += "</table>"

In [98]:
wholetext = ""
wholetext += initial
for exid in experiments:
    wholetext += extexts[exid]
wholetext += closure
f = open("index.html", "w")
f.write(wholetext)
f.close()
print(wholetext)

<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <meta content="IE=edge" http-equiv="X-UA-Compatible">
    <meta content="width=device-width, initial-scale=1" name="viewport">
    <title>Emotion-Aware DialogTTS Demo Page</title>
    <link href="/EDTTS-Demo/statics/bootstrap-5.2.3-dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="/EDTTS-Demo/statics/my.css" rel="stylesheet">
  </head>
  <body>
    <div class="container">
      <div class="row">
        <div class="container pt-5 mt-5 shadow p-5 mb-5 bg-white rounded">
          <div class="text-center">
            <h2>Emotion-Aware DialogTTS: Generating Empathetic Speech Dialogs with Turn-Level Emotion Conditioning</h2>
            <br>
            <h5 class="mb-4">Sho Inoue<sup>1,*</sup>, Min-Jae Hwang<sup>1</sup></h5>
              <p>
                <sup>1</sup>FAIR at Meta<br>
                <sup>*</sup>Work done at Meta<br>
              </p>
          </div>
          <br>
          <figure class="text